# Домашнее задание 2. Бейзлайны и детерминированные алгоритмы item-item

[RecSys types](https://drive.google.com/file/d/10i5oN2Cw60oLMvnQ4SKFkkotnhRS7Q8L/view?usp=sharing)

In [288]:
#pip install implicit

In [331]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [332]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [333]:
data['week_no'].nunique()

95

In [334]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [337]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [338]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


### Train-test split

В рекомендательных системах корректнее использовать train-test split по времени, а не случайно  
Я возьму последние 3 недели в качестве теста

In [339]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [340]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

# 1. Бейзлайны

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [341]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [342]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### 1.1 Random recommendation

In [343]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [344]:
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [345]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 3.24 s, sys: 24.1 ms, total: 3.27 s
Wall time: 3.28 s


In [346]:
items

array([ 1004906,  1033142,  1036325, ..., 15722756, 17170636, 15716393])

In [347]:
result.head(2)

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1009850, 8157053, 117092, 9297602, 13417002]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5591318, 13512102, 550183, 13213485, 12263788]"


### 1.2 Popularity-based recommendation

In [348]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    #print(recs)
    return recs.tolist()

In [349]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 148 ms, sys: 26.4 ms, total: 175 ms
Wall time: 174 ms


In [350]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1009850, 8157053, 117092, 9297602, 13417002]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5591318, 13512102, 550183, 13213485, 12263788]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender

- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

*Пример*  
item_1 - 5, item_2 - 7, item_3 - 4  # / sum  
item_1 - 5 / 16, item_2 - 7 / 16, item_3 - 4 / 16

In [351]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [352]:
# считаем вес каждого товара - сумма sales_value
item_sales = data_train.groupby('item_id')['sales_value'].sum()
item_sales

item_id
25671       20.94
26081        0.99
26093        1.59
26190        1.54
26355        1.98
            ...  
17381856     0.00
17382205     7.99
17383227     4.49
17827644     2.50
17829232    11.98
Name: sales_value, Length: 86865, dtype: float64

In [353]:
# для логарифмирования не подходят нули
item_sales.min()

0.0

In [354]:
# заменим нули на 0.00000000001
item_sales.values[item_sales.values==0] = 0.00000000001

In [355]:
log_weights = np.log(item_sales)
log_weights

item_id
25671        3.041661
26081       -0.010050
26093        0.463734
26190        0.431782
26355        0.683097
              ...    
17381856   -25.328436
17382205     2.078191
17383227     1.501853
17827644     0.916291
17829232     2.483239
Name: sales_value, Length: 86865, dtype: float64

In [356]:
#уберем отрицательные
log_weights = log_weights - log_weights.min()

In [357]:
log_weights.sum()

3226462.738864591

In [358]:
log_weights.min()

0.0

In [359]:
log_weights_norm = log_weights/log_weights.sum()

In [360]:
log_weights_norm.sum()

0.9999999999999998

In [361]:
log_weights_norm

item_id
25671       0.000012
26081       0.000011
26093       0.000011
26190       0.000011
26355       0.000011
              ...   
17381856    0.000003
17382205    0.000011
17383227    0.000011
17827644    0.000011
17829232    0.000012
Name: sales_value, Length: 86865, dtype: float64

In [362]:
log_weights_norm.values

array([1.16843191e-05, 1.07384810e-05, 1.08853242e-05, ...,
       1.12070756e-05, 1.10255883e-05, 1.15112433e-05])

In [363]:
%%time

def weighted_random_recommendation(items, weight_list, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    #items = np.array(items)
    #weights_list = np.array(weight_list)
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, p=weight_list, replace=False)
    
    return recs.tolist()

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.3 µs


In [375]:
items = log_weights_norm.index
weights = log_weights_norm.values

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))

In [376]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13004478, 9527458, 1077643, 986150, 1049733]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]","[1074711, 8175914, 12524655, 491933, 940223]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[116917, 1029424, 9245375, 9859223, 6602557]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]","[1033833, 5133371, 13210008, 992475, 824080]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [368]:
import pickle

In [369]:
with open('predictions_basic1.pickle', 'rb') as handle:
    result = pickle.load(handle)
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13004478, 9527458, 1077643, 986150, 1049733]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[116917, 1029424, 9245375, 9859223, 6602557]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [ ]:
items = log_weights_norm.index
weights = log_weights_norm.values

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))

In [323]:
#pip install ipynb

In [372]:
from ipynb.fs.full.my_metrics import precision_at_k

In [377]:
%%time

metrics = {}

for i in result.columns[1:]:
    metrics[i] = result.apply(lambda row: precision_at_k(row[i], row['actual'],5), axis=1).mean()

ipynb.fs.full.my_metrics:19: RuntimeWarning: invalid value encountered in long_scalars


CPU times: user 1.35 s, sys: 6.18 ms, total: 1.35 s
Wall time: 1.35 s


In [378]:
df = pd.DataFrame(metrics.values(), index = metrics.keys(), columns=['precision_at_k'])
df = df.sort_values(['precision_at_k'])
df

,precision_at_k
weighted_random_recommendation,0.000294
random_recommendation,0.000784
cosine,0.132909
itemitem,0.136925
popular_recommendation,0.155240
tfidf,0.160921
own_purchases,0.201917
actual,1.000000


**плохо показал себя weighted_random_recommendation, получается, что прямо пропорционально полулярности - это еще менее эффективно, чем просто случайный выбор**

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

### 1.1 Random recommendation top 5000

In [343]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [344]:
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [399]:
items = data_train.item_id.unique()
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [400]:
result['random_recommendation_5K'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))

In [401]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5K
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13004478, 9527458, 1077643, 986150, 1049733]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]","[1074711, 8175914, 12524655, 491933, 940223]","[933293, 12425418, 953539, 1123958, 1092424]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[116917, 1029424, 9245375, 9859223, 6602557]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]","[1033833, 5133371, 13210008, 992475, 824080]","[1127264, 5574377, 1044971, 927110, 13003092]"


### 1.2 Popularity-based recommendation

In [ ]:
#здесь нельзя улучшить по топ 5000, так как иы делаем тоже самое, только выбираем не 5000, а 5 самых популярных просто берем

In [348]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    #print(recs)
    return recs.tolist()

### 1.3 Weighted random recommender топ 5000

In [408]:
# считаем вес каждого товара - сумма sales_value
item_sales = data_train.groupby('item_id')['sales_value'].sum()
item_sales_sorted = item_sales.sort_values(axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last', 
                       ignore_index=False, key=None)
item_sales_5000 = item_sales_sorted.head(5000)
item_sales_5000

item_id
6534178     447799.94
6533889      40483.34
1029743      35764.66
6534166      30170.77
1082185      26029.96
              ...    
12188446       265.29
1112387        265.26
981716         265.17
5575861        265.14
915174         265.13
Name: sales_value, Length: 5000, dtype: float64

In [409]:
# нулей нет, отлично
item_sales_5000.min()

265.13

In [413]:
log_weights_5000 = np.log(item_sales_5000)
log_weights_5000

item_id
6534178     13.012102
6533889     10.608646
1029743     10.484716
6534166     10.314629
1082185     10.167003
              ...    
12188446     5.580824
1112387      5.580710
981716       5.580371
5575861      5.580258
915174       5.580220
Name: sales_value, Length: 5000, dtype: float64

In [414]:
log_weights_5000_norm = log_weights_5000/log_weights_5000.sum()

In [415]:
log_weights_5000_norm.sum()

1.0

In [418]:
items_5000 = log_weights_5000_norm.index
weights_5000 = log_weights_5000_norm.values
result['weighted_random_recommendation_5К'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_5000, weights_5000, n=5))



In [419]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5K,weighted_random_recommendation_5К
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13004478, 9527458, 1077643, 986150, 1049733]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]","[986947, 12807936, 834102, 1111270, 862185]","[933293, 12425418, 953539, 1123958, 1092424]","[1124323, 823862, 1122568, 937526, 1063739]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[116917, 1029424, 9245375, 9859223, 6602557]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]","[924192, 15595598, 5720633, 9419082, 13510931]","[1127264, 5574377, 1044971, 927110, 13003092]","[1005911, 841762, 880469, 847790, 999391]"


In [420]:
%%time

metrics = {}

for i in result.columns[1:]:
    metrics[i] = result.apply(lambda row: precision_at_k(row[i], row['actual'],5), axis=1).mean()

ipynb.fs.full.my_metrics:19: RuntimeWarning: invalid value encountered in long_scalars


CPU times: user 1.75 s, sys: 25.1 ms, total: 1.77 s
Wall time: 1.79 s


In [421]:
df = pd.DataFrame(metrics.values(), index = metrics.keys(), columns=['precision_at_k'])
df = df.sort_values(['precision_at_k'])
df

,precision_at_k
weighted_random_recommendation,0.000588
random_recommendation,0.000784
weighted_random_recommendation_5К,0.006366
random_recommendation_5K,0.006366
cosine,0.132909
itemitem,0.136925
popular_recommendation,0.155240
tfidf,0.160921
own_purchases,0.201917
actual,1.000000


## Улучшим ItemItemRecommender, выбирая число соседей K.

In [426]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [427]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', 
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [428]:
sparse_user_item

<2499x5001 sparse matrix of type '<class 'numpy.float64'>'
	with 667080 stored elements in Compressed Sparse Row format>

In [429]:
user_item_matrix.shape

(2499, 5001)

In [430]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [431]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [432]:
user_item_matrix

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [437]:
%%time

for i in range(1, 11):

    model = ItemItemRecommender(K=i, num_threads=3) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)
    
    result['itemitem_' + str(i) + 'k'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

...

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 18.1 s, sys: 191 ms, total: 18.3 s
Wall time: 18.3 s


In [438]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5K,...,itemitem_1k,itemitem_2k,itemitem_3k,itemitem_4k,itemitem_5k,itemitem_6k,itemitem_7k,itemitem_8k,itemitem_9k,itemitem_10k
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13004478, 9527458, 1077643, 986150, 1049733]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]","[986947, 12807936, 834102, 1111270, 862185]","[933293, 12425418, 953539, 1123958, 1092424]",...,"[999999, 1082185, 1029743, 995785, 1004906]","[999999, 1082185, 995242, 1029743, 840361]","[999999, 1082185, 981760, 995242, 1029743]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[116917, 1029424, 9245375, 9859223, 6602557]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]","[924192, 15595598, 5720633, 9419082, 13510931]","[1127264, 5574377, 1044971, 927110, 13003092]",...,"[999999, 1082185, 1098066, 6534178, 1127831]","[999999, 1082185, 1098066, 6534178, 826249]","[999999, 1082185, 981760, 1098066, 6534178]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]"


In [439]:
metrics = {}

for i in result.columns[1:]:
    metrics[i] = result.apply(lambda row: precision_at_k(row[i], row['actual'],5), axis=1).mean()

ipynb.fs.full.my_metrics:19: RuntimeWarning: invalid value encountered in long_scalars


In [440]:
df = pd.DataFrame(metrics.values(), index = metrics.keys(), columns=['precision_at_k'])
df = df.sort_values(['precision_at_k'])
df

,precision_at_k
weighted_random_recommendation,0.000588
random_recommendation,0.000784
weighted_random_recommendation_5К,0.006366
random_recommendation_5K,0.006366
cosine,0.132909
itemitem,0.136925
itemitem_5k,0.136925
itemitem_6k,0.142018
itemitem_7k,0.144858
itemitem_4k,0.144956


**лучший precision у модели с К=2 и 3**